# Answering Business Questions Using SQL

In this project, we'll use SQL to answer some typical business questions using the Chinook database.

A copy of the `chinook.db` schema can be seen below:

![chinook-schema](chinook-schema.svg)

### Setup

You can use the following code if you need to install `ipython-sql`:

    !conda install -yc conda-forge ipython-sql
    
First, we'll connect our Jupyter Notebook to our database file.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Next, let's take a first look at the database and list out the tables and views to get a better idea of what we're working with.

In [2]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Here is our scenario:

The Chinook Record Store has signed a deal with a new record label, and we need to select the first 3 out of 4 albums that we will add to the store. We only have the artists names and genre of music they produce as follows:


| Artist Name | Genre |
| :--- | --- |
| Regal | Hip-Hop |
|Red Tone | Punk |
|Meteor and the Girls | Pop |
|Slim Jim Bites | Blues |


### Selecting Albums to Purchase

First, we'll find out which genres sell best in the US to help us effectively choose which albums to advertise. 

In [3]:
%%sql

WITH usa_sold AS
   (SELECT il.*
      FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     WHERE i.billing_country = 'USA'
   )

SELECT g.name genre,
       COUNT(us.quantity) tracks_sold,
       ROUND(CAST(COUNT(us.quantity) AS FLOAT) / (
           SELECT COUNT(*) FROM usa_sold
       ) * 100, 1) percentage_sold
  FROM usa_sold us  
 INNER JOIN track t ON t.track_id = us.track_id
 INNER JOIN genre g ON g.genre_id = t.genre_id
 GROUP BY genre
 ORDER BY tracks_sold DESC
 LIMIT 15;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Pop,22,2.1
Latin,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


Looking at the best selling genres in the US we determine that we should purchase 

| Artist Name | Genre | % Sold |
| :--- | --- | --- |
|Red Tone | Punk | 12.4 |
|Slim Jim Bites | Blues | 3.4 |
|Meteor and the Girls | Pop | 2.1 |

We will not purchase Regal's Hip Hop album since Hip Hop/Rap only makes up 1.9% of sales by genre.

Something to note however is that Rock makes up a massive 53.4% of albums sold, so we should be on the lookout for any new artists in this genre.


### Analyzing Employee Sales Performance


Next, we'll look at the purchases of customers to see which of the sales support agents are the best performers.

In [11]:
%%sql

WITH customer_support_rep AS
     (
     SELECT i.customer_id,
            c.support_rep_id,
            SUM(i.total) AS total
       FROM invoice i
      INNER JOIN customer c ON i.customer_id = c.customer_id
      GROUP BY i.customer_id, c.support_rep_id
     )
    
SELECT 
       e.first_name || " " || e.last_name AS employee,
       e.hire_date,
       CAST(SUM(csr.total) AS INT) AS total_sales
  FROM customer_support_rep csr
 INNER JOIN employee e ON e.employee_id = csr.support_rep_id
 GROUP BY employee;

 * sqlite:///chinook.db
Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731
Margaret Park,2017-05-03 00:00:00,1584
Steve Johnson,2017-10-17 00:00:00,1393


There doesn't seem to be a significant difference between the top-performing employee, and the lowest-performing employee, given that the total sales of each employee seems to roughly correspond to how long they've been employed.

### Analyzing Sales by Country

Country, Customers, Total Sales, Average Order Amount, Customer Lifetime Value

Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis.

Write a query that collates data on purchases from different countries.

Where a country has only one customer, collect them into an "Other" group.

The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.

For each country, include:
total number of customers
total value of sales
average value of sales per customer
average order value

In [1]:
%%sql

invoice.billing_country

SELECT country,
       customers,
       total_sales,
       average_order,
       customer_lifetime_value
FROM


UsageError: Cell magic `%%sql` not found.


Findings

### Albums vs Individual Tracks

Findings.

The idea for this project comes from the [DATAQUEST](https://app.dataquest.io/) **Intermediate SQL for Data Analysis** course.